In [1]:
import gymnasium as gym
from stk_actor.wrappers import StuckStopWrapper
import torch
import torch.nn.functional as F

path = 'stk_actor/trained_agents/'
agent = 'normed_a2c_num5_best'

class PreprocessObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        """
        A Gym wrapper to preprocess mixed observation space (continuous + discrete)
        into a flat tensor.
        
        Args:
            env: The Gym environment to wrap.
        """
        super().__init__(env)
        self.observation_space = self._get_flat_observation_space(env.observation_space)
        self.mean = torch.load(path+f'{agent}/buffer_mean', map_location='cpu')
        self.std = torch.load(path+f'{agent}/buffer_std', map_location='cpu')

    def _get_flat_observation_space(self, observation_space):
        """
        Create a flat observation space based on the original observation space.
        
        Args:
            observation_space: Original observation space with 'continuous' and 'discrete' components.
        
        Returns:
            A flattened observation space.
        """
        continuous_dim = observation_space['continuous'].shape[0]
        discrete_dims = sum(space.n for space in observation_space['discrete'])
        flat_dim = continuous_dim + discrete_dims
        return gym.spaces.Box(low=-float('inf'), high=float('inf'), shape=(flat_dim,), dtype=float)

    def observation(self, obs):
        """
        Process the observation into a flat tensor.
        
        Args:
            obs: The raw observation from the environment.
        
        Returns:
            A preprocessed flat tensor.
        """
        continuous_obs, discrete_obs = obs['continuous'], obs['discrete']
        continuous_tensor = torch.FloatTensor(continuous_obs)
        
        discrete_tensors = [
            F.one_hot(torch.tensor(x), num_classes=num_classes.n).float()
            for x, num_classes in zip(discrete_obs, self.env.observation_space['discrete'])
        ]
        
        flat_tensor = torch.cat([continuous_tensor] + discrete_tensors)
        normed_flat_tensor = (flat_tensor - self.mean) / (self.std + 1e-8)
        return normed_flat_tensor

import gymnasium as gym
from gymnasium import Wrapper

class SkipFirstNStepsWrapper(Wrapper):
    def __init__(self, env, n):
        super().__init__(env)
        self.n = n

    def reset(self, **kwargs):
        # Reset the environment
        obs, info = self.env.reset(**kwargs)
        # Skip the first n steps
        for _ in range(self.n):
            obs, _, done, truncated, info = self.env.step(self.env.action_space.sample())
            if done or truncated:
                obs, info = self.env.reset(**kwargs)
        return obs, info

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from typing import Dict, List, Tuple, Union, Type
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

import gymnasium as gym
from gymnasium import spaces

def get_device(device: Union[torch.device, str] = "auto") -> torch.device:
    if device == "auto":
        device = "cuda"
    device = torch.device(device)
    if device.type == torch.device("cuda").type and not torch.cuda.is_available():
        return torch.device("cpu")
    return device

class BaseFeaturesExtractor(nn.Module):
    def __init__(self, observation_space: gym.Space, features_dim: int = 0) -> None:
        super().__init__()
        assert features_dim > 0
        self._observation_space = observation_space
        self._features_dim = features_dim
    @property
    def features_dim(self) -> int:
        return self._features_dim

def get_flattened_obs_dim(observation_space: spaces.Space) -> int:
    if isinstance(observation_space, spaces.MultiDiscrete):
        return sum(observation_space.nvec)
    else:
        return spaces.utils.flatdim(observation_space)

class FlattenExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space) -> None:
        super().__init__(observation_space, get_flattened_obs_dim(observation_space))
        self.flatten = nn.Flatten()
    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.flatten(observations)
    
class MlpExtractor(nn.Module):
    def __init__(
        self,
        feature_dim: int,
        net_arch: Union[List[int], Dict[str, List[int]]],
        activation_fn: Type[nn.Module],
        device: Union[torch.device, str] = "auto",
    ) -> None:
        super().__init__()
        # device = torch.get_device(device)
        policy_net: List[nn.Module] = []
        value_net: List[nn.Module] = []
        last_layer_dim_pi = feature_dim
        last_layer_dim_vf = feature_dim

        if isinstance(net_arch, dict):
            pi_layers_dims = net_arch.get("pi", []) 
            vf_layers_dims = net_arch.get("vf", []) 
        else:
            pi_layers_dims = vf_layers_dims = net_arch
        for curr_layer_dim in pi_layers_dims:
            policy_net.append(nn.Linear(last_layer_dim_pi, curr_layer_dim))
            policy_net.append(activation_fn())
            last_layer_dim_pi = curr_layer_dim
        for curr_layer_dim in vf_layers_dims:
            value_net.append(nn.Linear(last_layer_dim_vf, curr_layer_dim))
            value_net.append(activation_fn())
            last_layer_dim_vf = curr_layer_dim

        self.latent_dim_pi = last_layer_dim_pi
        self.latent_dim_vf = last_layer_dim_vf
        self.policy_net = nn.Sequential(*policy_net)#.to(device)
        self.value_net = nn.Sequential(*value_net)#.to(device)

    def forward(self, features: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        :return: latent_policy, latent_value of the specified network.
            If all layers are shared, then ``latent_policy == latent_value``
        """
        return self.forward_actor(features), self.forward_critic(features)

    def forward_actor(self, features: torch.Tensor) -> torch.Tensor:
        return self.policy_net(features)

    def forward_critic(self, features: torch.Tensor) -> torch.Tensor:
        return self.value_net(features)

    
class Policy(nn.Module):
    def __init__(self, observation_space, action_dims, net_arch, activation_fn,):
        super().__init__()
        self.features_extractor = FlattenExtractor(observation_space)
        self.pi_features_extractor = self.features_extractor
        self.vf_features_extractor = self.features_extractor
        self.mlp_extractor = MlpExtractor(
            self.features_extractor.features_dim,
            net_arch=net_arch,
            activation_fn=activation_fn,
        )
        self.action_net = nn.Linear(net_arch[-1], sum(action_dims))
        self.value_net = nn.Linear(net_arch[-1], 1)


class UnifiedSACPolicy(nn.Module):
    def __init__(self, observation_space, action_dims, net_arch, activation_fn):
        super().__init__()
        
        self.shared = Policy(
            observation_space,
            action_dims,
            net_arch=net_arch,
            activation_fn=activation_fn
        )
        self.action_dims = action_dims
    
    def forward(self, x):
        x = self.shared.features_extractor(x)
        x = self.shared.mlp_extractor.policy_net(x)
        x = self.shared.action_net(x)
        return x
    
    def sample(self, x, deterministic=False):
        logits = self.forward(x)
        
        # Split logits for each action dimension
        split_logits = torch.split(logits, self.action_dims, dim=-1)
        
        actions = []
        log_probs = []
        probs = []
        
        for logit in split_logits:
            distribution = Categorical(logits=logit)
            if deterministic:
                action = torch.argmax(logit, dim=-1)
            else:
                action = distribution.sample()
            
            log_prob = distribution.log_prob(action)
            prob = F.softmax(logit, dim=-1)
            
            actions.append(action)
            log_probs.append(log_prob)
            probs.append(prob)
        
        return (
            torch.stack(actions),
            torch.stack(log_probs),
            probs
        )
    
#policy = torch.load('policy_512_512_512_512_SiLU_3_statedict', map_location='cuda')


from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
import gymnasium as gym
from pystk2_gymnasium import AgentSpec
from bbrl.agents.gymnasium import ParallelGymAgent, make_env
from functools import partial

tracks = [
    'abyss',
    'black_forest',
    'candela_city',
    'cocoa_temple',
    'cornfield_crossing',
    'fortmagma',
    'gran_paradiso_island',
    'hacienda',
    'lighthouse',
    'mines',
    'minigolf',
    'olivermath',
    'ravenbridge_mansion',
    'sandtrack',
    'scotland',
    'snowmountain',
    'snowtuxpeak',
    'stk_enterprise',
    'volcano_island',
    'xr591',
    'zengarden',

# # #   ==================   #

#     'fortmagma',
#     'ravenbridge_mansion',
#     'snowmountain',
#     'cocoa_temple',
#     'sandtrack',    
#     'scotland', 
#     'stk_enterprise',
#     'volcano_island', # 1104
#     'xr591', # 864   
]


karts = [12]
n_envs = len(tracks)*len(karts)

print('making', n_envs, 'environments')
vec_env = make_vec_env(
    "supertuxkart/flattened_multidiscrete-v0",
    # seed=12,
    n_envs=n_envs, 
    wrapper_class=lambda x : (
        SkipFirstNStepsWrapper(
            StuckStopWrapper(
                PreprocessObservationWrapper(x),
                n=92,
            ), 
            n=20,
        )
    ), 
    env_kwargs={
    'render_mode':None, 'agent':AgentSpec(use_ai=False, name="walid"), #'track':'minigolf', 
    'laps':1,
    'difficulty':2, 
    'num_kart':12, #'difficulty':0
})

ix = 0
for num_kart in enumerate(karts):
    for track in enumerate(tracks):
        venv = vec_env.envs[ix]
        venv.env.default_track = track
        venv.env.num_kart = num_kart
        print(ix, track, )
        ix+=1

net_arch=[1024,1024,1024]
activation_fn=torch.nn.Tanh
filename = path+f'{agent}/statedict'

action_dims = [space.n for space in vec_env.action_space]
unified_policy = UnifiedSACPolicy(
    vec_env.observation_space, 
    action_dims, 
    net_arch=net_arch, 
    activation_fn=activation_fn
)
unified_policy.load_state_dict(torch.load(filename, map_location='cpu'))


making 21 environments
..:: Antarctica Rendering Engine 2.0 ::..


/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_42946/2426701993.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mean = torch.load(path+f'{agent}/buffer

0 (0, 'abyss')
1 (1, 'black_forest')
2 (2, 'candela_city')
3 (3, 'cocoa_temple')
4 (4, 'cornfield_crossing')
5 (5, 'fortmagma')
6 (6, 'gran_paradiso_island')
7 (7, 'hacienda')
8 (8, 'lighthouse')
9 (9, 'mines')
10 (10, 'minigolf')
11 (11, 'olivermath')
12 (12, 'ravenbridge_mansion')
13 (13, 'sandtrack')
14 (14, 'scotland')
15 (15, 'snowmountain')
16 (16, 'snowtuxpeak')
17 (17, 'stk_enterprise')
18 (18, 'volcano_island')
19 (19, 'xr591')
20 (20, 'zengarden')


/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_42946/2426701993.py:324: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unified_policy.load_state_dict(torch.load(f

<All keys matched successfully>

In [2]:
from stable_baselines3 import A2C

steps = [(
    # 1024*8,
    20*n_envs,
    300_000,
)]
for n_steps, total_timesteps in steps:
    # model = PPO(
    #     "MlpPolicy", 
    #     vec_env, 
    #     verbose=1, 
    #     policy_kwargs = dict(net_arch=net_arch, activation_fn=activation_fn,),
    #     device='cpu',
    #     learning_rate=0.0001,
    #     batch_size=128,
    #     n_epochs=100,
    #     n_steps=n_steps,
    #     tensorboard_log="./outputs/",
    #     # ent_coef=0.001,
    #     clip_range=0.2,
    # )
    model = A2C(
        "MlpPolicy", 
        vec_env, 
        verbose=1, 
        policy_kwargs = dict(net_arch=net_arch, activation_fn=activation_fn,),
        device='cpu',
        learning_rate=0.0006,
        n_steps=n_steps,
        tensorboard_log="./outputs/",
        use_rms_prop=False,
        normalize_advantage=True,
    )
    print('DOING', n_steps, total_timesteps)
    model.policy.load_state_dict(unified_policy.shared.state_dict())
    model.learn(total_timesteps=total_timesteps, progress_bar=True, log_interval=1)

        


Using cpu device
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
DOING 420 300000
Logging to ./outputs/A2C_12


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 348      |
|    ep_rew_mean     | 458      |
| time/              |          |
|    fps             | 56       |
|    iterations      | 1        |
|    time_elapsed    | 154      |
|    total_timesteps | 8820     |
---------------------------------


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 502      |
|    ep_rew_mean        | 520      |
| time/                 |          |
|    fps                | 54       |
|    iterations         | 2        |
|    time_elapsed       | 322      |
|    total_timesteps    | 17640    |
| train/                |          |
|    entropy_loss       | -0.222   |
|    explained_variance | 0.14     |
|    learning_rate      | 0.0006   |
|    n_updates          | 1        |
|    policy_loss        | -0.0102  |
|    value_loss         | 2.53e+03 |
------------------------------------


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 535      |
|    ep_rew_mean        | 499      |
| time/                 |          |
|    fps                | 51       |
|    iterations         | 3        |
|    time_elapsed       | 517      |
|    total_timesteps    | 26460    |
| train/                |          |
|    entropy_loss       | -0.244   |
|    explained_variance | 0.264    |
|    learning_rate      | 0.0006   |
|    n_updates          | 2        |
|    policy_loss        | -0.0199  |
|    value_loss         | 1.95e+03 |
------------------------------------


...

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 631      |
|    ep_rew_mean        | 468      |
| time/                 |          |
|    fps                | 54       |
|    iterations         | 31       |
|    time_elapsed       | 4973     |
|    total_timesteps    | 273420   |
| train/                |          |
|    entropy_loss       | -0.248   |
|    explained_variance | 0.288    |
|    learning_rate      | 0.0006   |
|    n_updates          | 30       |
|    policy_loss        | -0.00271 |
|    value_loss         | 1.8e+03  |
------------------------------------


-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 628       |
|    ep_rew_mean        | 478       |
| time/                 |           |
|    fps                | 54        |
|    iterations         | 32        |
|    time_elapsed       | 5148      |
|    total_timesteps    | 282240    |
| train/                |           |
|    entropy_loss       | -0.257    |
|    explained_variance | 0.458     |
|    learning_rate      | 0.0006    |
|    n_updates          | 31        |
|    policy_loss        | -0.000971 |
|    value_loss         | 1.66e+03  |
-------------------------------------


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 620      |
|    ep_rew_mean        | 480      |
| time/                 |          |
|    fps                | 54       |
|    iterations         | 33       |
|    time_elapsed       | 5372     |
|    total_timesteps    | 291060   |
| train/                |          |
|    entropy_loss       | -0.238   |
|    explained_variance | 0.318    |
|    learning_rate      | 0.0006   |
|    n_updates          | 32       |
|    policy_loss        | 0.0149   |
|    value_loss         | 1.63e+03 |
------------------------------------


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 642      |
|    ep_rew_mean        | 529      |
| time/                 |          |
|    fps                | 53       |
|    iterations         | 34       |
|    time_elapsed       | 5583     |
|    total_timesteps    | 299880   |
| train/                |          |
|    entropy_loss       | -0.253   |
|    explained_variance | 0.2      |
|    learning_rate      | 0.0006   |
|    n_updates          | 33       |
|    policy_loss        | 0.0188   |
|    value_loss         | 2.5e+03  |
------------------------------------


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 639      |
|    ep_rew_mean        | 542      |
| time/                 |          |
|    fps                | 53       |
|    iterations         | 35       |
|    time_elapsed       | 5730     |
|    total_timesteps    | 308700   |
| train/                |          |
|    entropy_loss       | -0.241   |
|    explained_variance | 0.188    |
|    learning_rate      | 0.0006   |
|    n_updates          | 34       |
|    policy_loss        | 0.0218   |
|    value_loss         | 1.91e+03 |
------------------------------------


In [3]:
model.save(f'final_a2c_{n_steps}')
f'final_a2c_{n_steps}'

'final_a2c_420'